![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [47]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

cc_apps.info()
#From first inspection there are no null data points 

cc_apps.replace("?", np.nan, inplace=True)

clean_data = cc_apps.copy()

# Convert column index to string to match the key used in the code
clean_data.columns = clean_data.columns.astype(str)

# Fill NaN values in column "2" with the mean of the column
clean_data["2"] = clean_data["2"].astype(float)  # Ensure the column is of type float
clean_data["2"].fillna(clean_data["2"].mean(), inplace=True)

clean_data["2"]

# Fixing the loop and conditions
for x in clean_data.columns:
    if clean_data[x].dtype == "object":
        clean_data[x].fillna(clean_data[x].mode()[0], inplace=True)
    else:
        clean_data[x].fillna(clean_data[x].mean(), inplace=True)
        
# One-hot encoding 
clean_data = pd.get_dummies(clean_data, drop_first=True)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB


In [46]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Assuming clean_data is a DataFrame and the last column is the target variable
Target_Variable = clean_data.iloc[:, -1].values
Features = clean_data.iloc[:, :-1].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(Features, Target_Variable, test_size=0.30, random_state=42)

scaler = StandardScaler()

# Corrected the method calls for scaling
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

log = LogisticRegression()

# Corrected the fit method call to include both X_train and y_train
log.fit(X_train, y_train)

# Corrected the predict method call to use X_test
y_pred = log.predict(X_test)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

# Define the parameter grid for GridSearchCV
params = {
    'C': [0.1, 1, 10, 100],
    'solver': ['newton-cg', 'lbfgs', 'liblinear']
}

grid_search = GridSearchCV(estimator=log, param_grid=params, cv=4)
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print("Best parameters found: ", grid_search.best_params_)
print("Best estimator found: ", grid_search.best_estimator_)

best_score = grid_search.score(X_test, y_test)
print("Best score: ", best_score)

[[75 22]
 [19 91]]
Best parameters found:  {'C': 1, 'solver': 'newton-cg'}
Best estimator found:  LogisticRegression(C=1, solver='newton-cg')
Best score:  0.8019323671497585
